In [1]:
import pandas as pd
df=pd.read_csv("Reviews.csv")

In [2]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
df=df[:100]

In [4]:
import re,nltk
from string import digits
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words=set(stopwords.words('english'))
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [5]:
def clean(text):
    text=re.sub('[^a-zA-Z]', " ", str(text))
    text=text.lower()
    text_tokens=word_tokenize(text)
    text_tokens_no_sw=[w for w in text_tokens if not w in stop_words]
    text=TreebankWordDetokenizer().detokenize(text_tokens_no_sw)
    return text

In [6]:
df['clean_text']=df.Text.apply(clean)

In [7]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,clean_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanuts p...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confection around centuries light pillowy citr...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...


In [8]:
df1=df.drop('Text',1)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer 
model=TfidfVectorizer()

In [10]:
core=model.fit_transform(df1['clean_text'])

In [11]:
final=pd.DataFrame(core.toarray(),columns=model.get_feature_names())

In [12]:
final.head()

,abby,abdominal,able,absence,according,account,across,activate,actual,actually,...,www,year,years,yet,young,yrs,yum,yummy,zero,zip
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.186814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.129460,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.224794,0.0,0.0


In [13]:
import numpy as np
from sklearn.decomposition import NMF

In [17]:
nmf=NMF(4,random_state=42,alpha=0.1,l1_ratio=0.5,init='nndsvd')
arr=nmf.fit_transform(final)
nmf.components_

array([[0.01990615, 0.        , 0.02593713, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02900844, 0.        ,
        0.        ]])

In [18]:
def display_topics(H,feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( H[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])


In [29]:
df1.apply(display_topics(nmf.components_,model.get_feature_names(),df1['clean_text'],10000,1000))

Topic 0:
br oatmeal mccann instant sugar pack variety good product regular great cinnamon brown maple apples better morning like oats amazon water almost boxes count best excellent irish time brand tastes make flavors steel every thing taste store cut really convenient meal thanks happy use price grocery minutes href www http gp com half need plus prepare bowl milk flavor per well food healthy even three likes box texture instead could breakfast able minute though brands get add microwave apple corn cane cooks syrup little save may high non ordering find one celiac disease abby lifesaver give take coffee wife eo qw reccomended watch maybe getting gum guar ounce fat hardy post folks fiber palatable digestible bloat surgery bariatric easily fan try sweet low thought ordered health regularly keeps cook earth quality longer daughter much found offer staral oat soggy close hits holds become sure anything often thick cold actually eaten quick powder boiling bit followed selling love say seco

TypeError: ("'NoneType' object is not callable", 'occurred at index Id')

In [30]:
from sklearn.decomposition import LatentDirichletAllocation
lda=LatentDirichletAllocation()
fi=lda.fit_transform(final)

In [39]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [42]:
final.apply(print_topics(lda,model,9))


Topic #0:
br hot shared coffee less slightly delivered good always

Topic #1:
enjoyed order tea oatmeal sweet know tastes since like

Topic #2:
flavors buying satisfying food good chewy taffy soft eat

Topic #3:
dog around okay buy food way go twizzlers severs

Topic #4:
food product better fast bad great family dog like

Topic #5:
taffy product candy great price like red love br

Topic #6:
oatmeal instant br food mccann sugar dog foods natural

Topic #7:
food sauce br allergies dog hot dogs love soft

Topic #8:
br pack twizzlers instant variety product mccann bags http

Topic #9:
eat delicious days little oatmeal much arrived purchased flavor


TypeError: ("'NoneType' object is not callable", 'occurred at index abby')